In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,TimeoutException,StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import os
import pandas as pd

In [12]:
# Khởi tạo trình duyệt
options = Options()
options.add_argument("--start-maximized")
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

In [ ]:
# Truy cập vào trang web
base_url = "https://nhathuoclongchau.com.vn/"
driver.get(base_url)
wait = WebDriverWait(driver,15)

wait.until(EC.presence_of_all_elements_located((By.TAG_NAME,"body")))

# Navigate to thuốc section
thuoc_menu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='/thuoc']")))
driver.execute_script("arguments[0].click();", thuoc_menu)
time.sleep(2)

# Go to tra cứu thuốc
driver.execute_script("window.scrollBy(0,900);")
time.sleep(2)
tracuu = driver.find_element(By.CSS_SELECTOR, "a[href='/thuoc/tra-cuu-thuoc-a-z']")
driver.execute_script("arguments[0].click();", tracuu)
#wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"div.pagination")))
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"div.no-scrollbar")))

# Tìm danh sách các ký tự từ A-Z và 5
char_buttons = driver.find_elements(By.CSS_SELECTOR, "div.no-scrollbar div.cursor-pointer")
alphabets = [btn.text.strip() for btn in char_buttons if btn.text.strip()]

thuoc_data = []

for alpha in alphabets:
    try:
        print(f"\n Đang xử lý danh mục: {alpha}")
        page_num = 1 
        while True:
            url = f"https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z?alphabet={alpha}&page={page_num}"
            driver.get(url)
        
            try:
                wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"div.medicine-item")))
                print(f"Trang {page_num} - {driver.current_url}")

                items = driver.find_elements(By.CSS_SELECTOR,"div.medicine-item a[href*='/thuoc/']")
            if not items:
                print(" - Không tìm thấy thuốc nào, dừng lại.")
                break
            for el in items:
                name = el.text.strip()
                link = el.get_attribute("href")
                thuoc_data.append({"Tên thuốc": name, "Link": link})
            # Kiểm tra nếu có trang tiếp theo
            next_btn = driver.find_elements(By.CSS_SELECTOR,"ul.pagination li a[rel='next']")
            if next_btn:
                page += 1
                time.sleep(1)
            else:
                break
            except TimeoutException:
                print("Time out -> bỏ quan trang")
                break
    

driver.quit()



In [14]:
df_thuoc = pd.DataFrame(thuoc_data)
print(f"\n Tổng số link thuốc thu được: {len(df_thuoc)}")
df_thuoc.head()


 Tổng số link thuốc thu được: 0


""


In [ ]:
# 🧱 Cài đặt thư viện cần thiết
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import pandas as pd
import time

# 🚀 Cấu hình WebDriver cho Jupyter Notebook
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

# 🏁 Bắt đầu tại trang tra cứu A-Z
start_url = "https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z"
driver.get(start_url)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.no-scrollbar")))

# 🔍 Tìm danh sách các ký tự A-Z và 5
char_buttons = driver.find_elements(By.CSS_SELECTOR, "div.no-scrollbar div.cursor-pointer")

# 🧺 Lưu kết quả
thuoc_data = []

# 🔁 Lặp qua từng ký tự
for i in range(len(char_buttons)):
    try:
        char_buttons = driver.find_elements(By.CSS_SELECTOR, "div.no-scrollbar div.cursor-pointer")
        char_button = char_buttons[i]
        char = char_button.text.strip()
        if not char:
            continue

        print(f"\n🔎 Đang xử lý danh mục: {char}")
        driver.execute_script("arguments[0].click();", char_button)
        time.sleep(1)
        current_url = driver.current_url

        # 🔁 Phân trang trong từng danh mục
        page_num = 1
        while True:
            try:
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.medicine-item")))
                print(f"  📄 Trang {page_num} - {driver.current_url}")

                # 🧼 Lấy các thuốc trên trang hiện tại
                items = driver.find_elements(By.CSS_SELECTOR, "div.medicine-item a[href*='/thuoc/']")
                for el in items:
                    name = el.text.strip()
                    link = el.get_attribute("href")
                    thuoc_data.append({"Tên thuốc": name, "Link": link})

                # ➡️ Tìm nút "Trang tiếp" (nếu có)
                next_button = driver.find_elements(By.CSS_SELECTOR, "ul.pagination li a[rel='next']")
                if next_button and next_button[0].is_enabled():
                    driver.execute_script("arguments[0].click();", next_button[0])
                    page_num += 1
                    time.sleep(1)
                else:
                    break
            except TimeoutException:
                print("  ⚠️ Timeout - bỏ qua trang.")
                break

    except StaleElementReferenceException:
        print("⚠️ Element bị stale, bỏ qua danh mục này.")
        continue

# ✅ Đóng trình duyệt sau khi hoàn tất
driver.quit()

# 📊 Hiển thị kết quả bằng pandas
df_thuoc = pd.DataFrame(thuoc_data)
print(f"\n✅ Tổng số thuốc thu được: {len(df_thuoc)}")
df_thuoc.head()


In [4]:
with open("danh_muc_links\danh_muc_links.txt","w",encoding="utf-8") as f:
    for name, link in ds_thuoc:
        f.write(f"{link}\n")

print("Đã lưu danh sách thành công")

Đã lưu danh sách thành công


<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Admin\AppData\Local\Temp\ipykernel_23984\4071047285.py:1: SyntaxWarning: invalid escape sequence '\d'
  with open("danh_muc_links\danh_muc_links.txt","w",encoding="utf-8") as f:


In [3]:
# Lấy danh sách link tất cả các thuốc từ các link danh mục
file_path = os.path.join("danh_muc_links","danh_muc_links.txt")

with open(file_path,"r",encoding="utf-8") as f:
    danh_muc_links = [line.strip() for line in f.readlines() if line.strip()]



In [4]:
all_thuoc_links = []

for link in danh_muc_links:
    print(f"Đang truy cập vào danh mục: {link}")
    driver.get(link)
    time.sleep(3)


while True:
    thuoc_links = driver.execute_script(
        """
        return Array.from(document.querySelectorAll("a[href^='/thuoc/']"))
            .map(link => link.href)
            .filter(href => !href.includes('tra-cuu-thuoc-a-z') && href.includes('/thuoc/'));
"""
    )
    all_thuoc_links.extend(thuoc_links)


    #Kiểm tra có nút sang trang tiếp không
    try:
        current_page = driver.find_element(By.CSS_SELECTOR,'li.flex.items-center.justify-center.cursor-default')
        next_li = current_page.find_element(By.XPATH,'following-sibling::li[1]')
        driver.execute_script("arguments[0].click();",next_li)
        time.sleep(2)
    except NoSuchElementException:
        print("Không còn trang kế tiếp")
        break
    
    

Đang truy cập vào danh mục: https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z
Đang truy cập vào danh mục: https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z
Đang truy cập vào danh mục: https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z?alphabet=A
Đang truy cập vào danh mục: https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z?alphabet=B
Đang truy cập vào danh mục: https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z?alphabet=C
Đang truy cập vào danh mục: https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z?alphabet=D
Đang truy cập vào danh mục: https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z?alphabet=E
Đang truy cập vào danh mục: https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z?alphabet=F
Đang truy cập vào danh mục: https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z?alphabet=G
Đang truy cập vào danh mục: https://nhathuoclongchau.com.vn/thuoc/tra-cuu-thuoc-a-z?alphabet=H
Đang truy cập vào danh mục: https://nhathuoclongchau.com.vn/thuoc/tra-cu

In [6]:
#Xóa trùng link(nếu có)
all_thuoc_links = list(set(all_thuoc_links))

#Lưu file
with open("danh_muc_links\DS_thuoc_links.txt","w",encoding="utf-8") as f:
    for link in all_thuoc_links:
        f.write(link + "\n")

print(f"Đã thu thập {len(all_thuoc_links)} link thuốc")

Đã thu thập 17 link thuốc


<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Admin\AppData\Local\Temp\ipykernel_22284\1120402743.py:5: SyntaxWarning: invalid escape sequence '\D'
  with open("danh_muc_links\DS_thuoc_links.txt","w",encoding="utf-8") as f:
